# EEG Explorer

In [1]:
import os, sys
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import plotly.express as px

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### Importação dos arquivos do bucket

### Verificação de importação dos arquivos

In [2]:
# TROCAR CAMINHO DE IMPORTAÇÃO
files_path = './dataset_eeg_cafe2022/RenamedFiles'
filtered_path = './dataset_eeg_cafe2022/filtered'

files = [file for file in os.listdir(files_path) if '.csv' in file]

if len(files) == 0:
    print('Sem dados no diretório especificado.')
    exit()

#### Visualização de um sample de dados

In [3]:
df = pd.read_csv(f'{files_path}/{files[0]}', delimiter='\t')
df.head()

,Unnamed: 0,Index,Fp1,Fp2,C3,C4,P7,P8,O1,O2,...,other.6,other.7,other.8,other.9,other.10,other.11,other.12,Timestamp,other.13,Timestamp (Formatted)
0,0,0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.671202e+09,0.0,2022-12-16 11:45:15.399844
1,1,1.0,-892.959792,-2575.611117,-59412.195156,-28126.087337,-54820.589281,-50429.502964,34704.779130,-49923.431658,...,28.0,48.0,6.0,208.0,0.0,0.0,0.0,1.671202e+09,0.0,2022-12-16 11:45:15.414879
2,2,2.0,-118653.533959,-119449.559252,-134480.911570,-136007.076560,-135834.635872,-136506.671653,-126563.015327,-135598.137172,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.671202e+09,0.0,2022-12-16 11:45:15.418877
3,3,3.0,35862.506208,30449.633309,-1073.514067,-18095.183592,-9108.799127,-13652.952710,-78267.383638,-8280.723492,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.671202e+09,0.0,2022-12-16 11:45:15.422851
4,4,4.0,109025.026399,107067.654741,84123.371982,89738.807312,83770.910926,83991.194507,65839.146964,85645.905626,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.671202e+09,0.0,2022-12-16 11:45:15.426893


### Canais a serem utilizados

In [4]:
channels = df.columns.to_list()[2:10]
channels

['Fp1', 'Fp2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2']

### Filtrando os sinais recém baixados

In [5]:
from src.filter import DataFilter as dfr

dfs = [pd.read_csv(f'{files_path}/{file}', delimiter='\t') for file in files]

In [6]:
data_filter = dfr.DataFilter(dfs)
data_filter.set_initial_and_final_timestamps()
data_filter.remove_other_columns()
data_filter.write_filtered_csvs(files)

### Dataset exemplo depois do filtro aplicado

In [7]:
# Separador , no lugar do \t
filtered_dfs = [pd.read_csv(f'{filtered_path}/{file}', delimiter=',') for file in files]
filtered_dfs[0]

,Unnamed: 0,Index,Fp1,Fp2,C3,C4,P7,P8,O1,O2,Timestamp
0,0,0.0,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,0.000000
1,1,1.0,-892.959792,-2575.611117,-59412.195156,-28126.087337,-54820.589281,-50429.502964,34704.779130,-49923.431658,0.015035
2,2,2.0,-118653.533959,-119449.559252,-134480.911570,-136007.076560,-135834.635872,-136506.671653,-126563.015327,-135598.137172,0.019033
3,3,3.0,35862.506208,30449.633309,-1073.514067,-18095.183592,-9108.799127,-13652.952710,-78267.383638,-8280.723492,0.023007
4,4,4.0,109025.026399,107067.654741,84123.371982,89738.807312,83770.910926,83991.194507,65839.146964,85645.905626,0.027049
...,...,...,...,...,...,...,...,...,...,...,...
18373,18373,4.0,-91838.406829,-94515.165373,-160530.727380,-144986.345947,-157730.831941,-151184.204260,-69126.135094,-152763.221085,74.707135
18374,18374,5.0,-76682.573263,-79147.826015,-93222.304153,-102578.620846,-99980.245620,-104608.346906,-155526.450821,-100957.817851,74.710165
18375,18375,6.0,134484.074794,129012.604783,99420.815690,92634.994465,90096.588504,85737.123484,-15200.752466,90955.665546,74.714159
18376,18376,7.0,114877.570781,113797.760722,85837.135363,99167.908985,89960.826669,95482.322886,115706.737330,93425.364302,74.718143


### Plotando os sinais

In [8]:
window_widget = widgets.IntSlider(min=1, max=1000, step=10, value=50)
file_widget = widgets.Combobox(options=files, value=files[0], description='Arquivo')
channel_widget = widgets.Dropdown(options=channels, value=channels[0], description='Canal')

In [9]:
def plot_signal(file, channel, window):
    delta = 30
    df = pd.read_csv(f'{filtered_path}/{file}', delimiter=',')
    timestamps = df['Timestamp'].to_numpy()
    channel_data = df[channel].to_numpy()
    moving_average = np.convolve(channel_data, np.ones(512)/512, mode='valid')
    first_derivative = 100*(moving_average[:-delta] - moving_average[delta:])/delta
    fig = px.line(x=timestamps[:len(first_derivative)], y=[channel_data[:len(first_derivative)], moving_average[:len(first_derivative)], first_derivative])
    fig.update_yaxes(range=[moving_average[10]-5E3, moving_average[10]+5E3])
    fig.show()

In [ ]:
out = widgets.interactive_output(plot_signal, 
                                 {'file': file_widget, 
                                  'channel': channel_widget, 
                                  'window': window_widget})

widgets.VBox([widgets.HBox([file_widget, channel_widget, window_widget]), out])